<a href="https://www.kaggle.com/code/rishabhsingh18/model-ark-experiment?scriptVersionId=187427548" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import torchvision.transforms import transforms
from torchvisions.datasets import ImageFolder
from torch.utils.data import DataLoader

# data preparation
def load_preprocess_data(data_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = ImageFolder(root=data_path, transform-transform)
    data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    return data_loader
# insert path to the datasets in ''
data_loader = load_preprocess_data('')